In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=false; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<b>Note:</b> the raw code for this notebook is by default hidden for easier reading.
To toggle the code on/off, click <a href="javascript:code_toggle()">here</a>.''')

# Setup

In [1]:
import torch
import Darwin as dw
import seaborn as sns

from torchvision import datasets, transforms
from bogus import atk
from ipywidgets  import interactive, fixed
from bokeh.io import output_notebook

output_notebook()
sns.set()

Loading BokehJS ...

In [2]:
dw.rebuild_from_save(0,generation=3,position=4,run=1)

{'nb_layers': 2, 'weight_decay': 1.9180935711755992e-05, 'lr': 0.028824730892500476, 'layers': [{'nb_units': 974, 'dropout_rate': 0.6113715592706362, 'activation': 'elu'}, {'nb_units': 612, 'dropout_rate': 0.46057154559085706, 'activation': 'tanh'}], 'optimizer': 'adam'}


NetFromBuildInfo(
  (model): Sequential(
    (flatten): Flatten()
    (fc_0): Linear(in_features=784, out_features=974, bias=True)
    (dropout_0): Dropout(p=0.6113715592706362)
    (elu0): ELU(alpha=1.0)
    (fc_1): Linear(in_features=974, out_features=612, bias=True)
    (dropout_1): Dropout(p=0.46057154559085706)
    (tanh1): Tanh()
    (logits): Linear(in_features=612, out_features=10, bias=True)
    (softmax): Softmax()
  )
)

In [ ]:
with open(r"D:\TELEGRAM_BOTS\NEURALUPDATES.txt") as file: #Credentials for telegram bot
    telegram_credentials = file.read().split('\n')

In [ ]:
transform = transforms.Compose([
                       transforms.ToTensor()])

MNIST_train = datasets.MNIST(r'D:\Data_sets/MNIST', 
                            train=True, download=True,
                            transform=transform)

MNIST_test = datasets.MNIST(r'D:\Data_sets/MNIST', 
                            train=False, download=True,
                            transform=transform)

train_loader = torch.utils.data.DataLoader(
                            MNIST_train, batch_size=64, 
                            shuffle=True, pin_memory=True)

test_loader = torch.utils.data.DataLoader(MNIST_test,
                            batch_size=1000, shuffle=True, 
                            pin_memory=True)

In [ ]:
NET_SPACE = {
    'nb_layers': {'lb': 1, 'ub': 3, 'mutate': 0.15},
    'lr': {'lb': 0.001, 'ub':0.1, 'mutate': 0.15},
    'weight_decay': {'lb': 0.00001, 'ub': 0.0004, 'mutate':0.2},
    'optimizer': {'func': ['sgd', 'adam', 'adadelta','rmsprop'], 'mutate': 0.2}
}

In [ ]:
LAYER_SPACE = {
    'nb_units':{'lb': 128, 'ub':2048, 'mutate': 0.15},
    'dropout_rate': {'lb': 0.0, 'ub': 0.7, 'mutate': 0.2},
    'activation': {'func': ['relu'], 'mutate':0.2}
                   #['linear','tanh','relu','sigmoid','elu'], 'mutate':0.2}
}

In [ ]:
atk_func = atk()

# Run 

In [ ]:
testing = dw.TournamentOptimizer(3, LAYER_SPACE, NET_SPACE, dw.randomize_network, 
                           dw.mutate_net, dw.NetFromBuildInfo, dw.train, dw.test,
                          train_loader, test_loader, atk_func.fgsm, telegram_credentials)

In [ ]:
testing.step(generations=1, epochs=1, save=True, phone=False)

# Plots

In [ ]:
dw.progressplotter(testing, clean=True)

In [ ]:
dw.progressplotter(testing)

In [ ]:
dw.diffplotter(testing)

In [ ]:
dw.bestplotter(testing, k=0)

In [ ]:
dw.avgplotter(testing)

In [ ]:
dw.best_printer(testing)

In [ ]:
dw.multi_plot(testing, test_loader, adv_func=atk_func.fgsm, adversarial=True, eps=0.5)

In [ ]:
dw.multi_plot(testing, test_loader, adversarial=False)

In [ ]:
df = dw.dataframer(testing)
#df.to_pickle(r"../data/neuroevolution4")

In [ ]:
pd.re

In [ ]:
opts = df.columns.tolist()[:-1] #last column contains dicts which causes bokeh to fail
gen_opts = ['all'] + df['Generation'].unique().tolist() 
y1_opts = opts[2:] + opts[:2]
y2_opts = opts[1:] + opts[:1]
interactive_plot = interactive(dw.int_plot, df=fixed(df), x=opts, y=y1_opts, x2=opts, y2=y2_opts, gen=gen_opts)

In [ ]:
interactive_plot #bokeh does not show on Github due to JS restrictions

In [ ]:
model = dw.rebuild_from_save(generation=28,)

In [ ]:
dw.multi_plot(testing, test_loader, adversarial=False, model=model)

In [ ]:
#dw.rebuild_from_save()

In [ ]:
model

In [ ]:
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
batch = next(iter(test_loader))
img = batch[0][0].data
label = batch[1][0]

In [ ]:
img.size()

In [ ]:
plt.imshow(img.numpy().reshape(28,28), cmap='Greys')

In [ ]:
label

In [ ]:
grads = {}

In [ ]:
def save_grad(name):  #closure for use as a hook in fgsm attack - otherwise gradients can't be obtained for images. 
    def hook(grad):
        grads[name] = grad
    return hook

In [ ]:
grads['grad'].size()

In [ ]:
x = Variable(img, requires_grad=True).cuda()

In [ ]:
x.register_hook(save_grad('grad'))

In [ ]:
h_adv = model(x)

In [ ]:
h_adv.shape

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
cost = criterion(h_adv, labels.cuda().view(1))

In [ ]:
cost.backward()

In [ ]:
h_adv

In [ ]:
h_adv[0].detach().cpu().numpy()

In [ ]:
np.exp(-215449.73)

In [ ]:
def fgsm(model, x, y, eps=0.3, x_val_min=0, x_val_max=1): #https://arxiv.org/pdf/1412.6572.pdf
        
    x_adv = Variable(x.data, requires_grad=True).cuda() #clean image
    x_adv.register_hook(self.save_grad('x_adv'))

    h_adv = model(x_adv) #clean pred

    cost = F.nll_loss(h_adv, y.cuda()) #negative log loss clean image and clean pred

    if x_adv.grad is not None:
        x_adv.grad.data.fill_(0)

    cost.backward()

    x_adv = x_adv + (eps*self.grads['x_adv'].sign())
    x_adv = torch.clamp(x_adv, x_val_min, x_val_max)


    return x_adv

In [ ]:
for data, target in test_loader:
    data, target = data.cuda(), target.cuda()
    data= adv_func(model, data, target, eps=eps)
    output = model(data)
    pred = output.max(1, keepdim=True)[1]
    correct += pred.eq(target.view_as(pred).cuda()).sum().item()
    test_loss += F.nll_loss(output, target, size_average=False).item()